Consolidate and export background and shading

In [1]:
%load_ext autoreload
%autoreload 2

import zarr
import dask.array as da
import pandas as pd
import numpy as np
from scipy import stats, ndimage

from pathlib import Path
from bsccm import BSCCM


home = str(Path.home())
data_root = home + '/leukosight_data/'

shading_file = np.load(file=data_root + 'shading_corrections.npz')
backgrounds = zarr.open(data_root + 'backgrounds.zarr' , mode='r')

fluor_channels = {'F1_BV711': 'Fluor_690-',
                'F2_BV650': 'Fluor_627-673',
                'F3_BV605': 'Fluor_585-625',
                'F4_BV570': 'Fluor_550-570',
                'F5_BV510': 'Fluor_500-550',
                'F6_BV421': 'Fluor_426-446'}

slidescan_db = pd.read_csv('slidescan_db.csv')



In [8]:
export_root = home + '/leukosight_data/BSCCM_extra/'
export_file = zarr.open(export_root + 'BSCCM_backgrounds_and_shading.zarr')

In [9]:
for i, (fluor_prefix, led_prefix) in enumerate(zip(['Slidescan 1-0_1', 'Slidescan 1-0_1', 'Slidescan 2-0b_1'],  
                                                 ['Slidescan 1-6_1', 'Slidescan 2-10a_1', 'Slidescan 2-0b_1'])):
    #Create flatfields
#     export_file.create_dataset(name='shading/{}/flatfield'.format(i), data=
#             shading_file[ff_prefix + '_ff_slide'])
#     export_file.create_dataset(name='shading/{}/background'.format(i), data=
#                                shading_file[ff_prefix + '_backgrounds'])

    #fluor backgrounds from closest unstained to group
    for fluor_channel in fluor_channels.keys():
        for percentile in [5, 10, 20, 40, 50]:
            export_file.create_dataset(name='unstained_backgrounds/{}/{}_{}'.format(i, fluor_channels[fluor_channel], percentile),
                data=backgrounds[fluor_prefix]['{}_{}_percentile'.format(fluor_channel, percentile)])
            print('{}           \r'.format(fluor_channel), end='')

    
    #per flatfield group led array backgrounds
    for led_array_channel in [
        'Brightfield', 'DF_50','DF_50_Bottom', 'DF_50_Right','DF_55','DF_60','DF_60_Bottom','DF_60_Right','DF_65',
        'DF_70','DF_70_Bottom','DF_70_Right','DF_75','DF_80','DF_80_Bottom','DF_80_Right', 'DF_85','DF_90',
        'DPC_Bottom','DPC_Left','DPC_Right','DPC_Top', 'LED119']:
        for percentile in [5, 10, 20, 40, 50]:
            export_file.create_dataset(name='led_array_backgrounds/{}/{}_{}'.format(
                i, led_array_channel, percentile),
                data=backgrounds[led_prefix]['{}_{}_percentile'.format(led_array_channel, percentile)])
        print('{}           \r'.format(led_array_channel), end='')


print('Shadings        ')
#per-slide fluorescence background 
for name in slidescan_db.Magellan:
    if type(name) is not str:
        continue
    if 'scan' not in name or name == 'Slidescan 1-10_1' or 'Slidescan 2-3 continued_1' == name :
        continue
    entry = slidescan_db[slidescan_db.Magellan == name]
    batch = int(entry.Batch) - 1
    antibodies = entry.Marker.astype(str).tolist()[0]
    replicate = 1 if ((int(entry.Index) == 24) or (int(entry.Index) == 26)) else 0
    print(batch, antibodies, replicate)

    
    for fluor_channel in fluor_channels.keys():
        for percentile in [5, 10, 20, 40, 50]:
            data = backgrounds[name]['{}_{}_percentile'.format(fluor_channel, percentile)]
            if name == 'Slidescan 2-0a':
                data = data[:, 196: 196 + 2056]
            export_file.create_dataset(name='per_slide_backgrounds/{}/{}/{}/{}_{}'.format(
                antibodies, batch, replicate, 
                fluor_channels[fluor_channel], percentile),
                data=data)
#         print('{}  {}          \r'.format(name, fluor_channels[fluor_channel]), end='')
            
        


Shadings               
0 CD3 0
0 CD19 0
0 CD56 0
0 CD123 0
0 HLA-DR 0
0 CD45 0
0 CD14 0
0 CD16 0
0 unstained 0
0 all 0
1 unstained 0
